In [30]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import HashingVectorizer


from nltk.tokenize import WordPunctTokenizer
from pymystem3 import Mystem

from tqdm import tqdm

In [56]:
data = pd.read_csv("/kaggle/input/train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [57]:
data.shape

(30000, 4)

In [58]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Другие способы лемматизации (pymorphy2, spaCy) (2 балла)

#### pymorphy2

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install pymorphy2-dicts
!pip install DAWG-Python

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
def preprocess_py(text: str) -> str:
    return pymorphy2_analyzer.parse(text.lower())[0].normal_form

def tokenize_X_py(X):
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            help_list = []
            for word in X[i][j].split(' '):
                word_py = preprocess_py(word)
                help_list.append(word_py)
            X[i][j] = help_list
    return X

In [ ]:
X_train_py = tokenize_X_py(X_train)
X_test_py = tokenize_X_py(X_test)

In [31]:
X_train_py

array([[list(['сапог', '', '', '46', '', '', 'размер', '', '', 'новый', '\n']),
        list(['сапог', '', '', '46', '', '', 'размер', '', '', 'новый', '\n'])],
       [list(['светильник', '', '', 'потолочный', '', '', 'swarovski', '\n']),
        list(['светильник', '', '', 'потолочный', '', '', 'swarovski', '', '', '6', '', '', 'штука', '', ',', '', 'цена', '', '', 'за', '', '', 'штука', '.', '', '', 'в', '', '', 'эксплуатация', '', '', '2', '', '', 'год', '', ',', '', 'продаваться', '', '', 'в', '', '', 'связь', '', '', 'с', '', '', 'смена', '', '', 'интерьер', '', '', 'в', '', '', 'квартира', '\n'])],
       [list(['iphone', '', '', '7', '', '', 'plus', '', '', '128gb', '', '', 'red', '', '', 'красный', '', '', 'в', '', '', 'наличие', '\n']),
        list(['\xa0/\n', '/\n', '', '', 'данный', '', '', 'цена', '', '', 'только', '', '', 'для', '', '', 'подписчик', '', '', 'instagram', ':', '', 'iqmac', '/\n', '/\n', '', '', 'новый', '', '', 'красный', '', '', 'айфон', '', '', '7', '', 

#### spaCy

In [38]:
!pip install spacy==2.1.9

     |████████████████████████████████| 30.8 MB 116 kB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 31.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 34.6 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 1.1 MB/s  eta 0:00:01
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
ERROR: en-core-web-lg 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Attempting uninstall: thinc
    Found existing installation: thinc 7.3.1
    Uninstalling thinc-7.3.1:
      Successfully uninstalled thinc-7.3.1
  Attempting uninstall: spacy
    Found existing installati

In [40]:
!pip install git+https://github.com/aatimofeev/spacy_russian_tokenizer

  Cloning https://github.com/aatimofeev/spacy_russian_tokenizer to /tmp/pip-req-build-cp398lsp
  Running command git clone -q https://github.com/aatimofeev/spacy_russian_tokenizer /tmp/pip-req-build-cp398lsp
  Created wheel for spacy-russian-tokenizer: filename=spacy_russian_tokenizer-0.1.1-py3-none-any.whl size=12674 sha256=110867d5512283c62a23aa7123e57d529379c75eec0b0b1698748d5b7622a987
  Stored in directory: /tmp/pip-ephem-wheel-cache-thysj_ee/wheels/c5/60/f5/9c8ef6b2fbe613a647d8099a6f5b0a0a8cad75c68c1d1e7f97
Successfully built spacy-russian-tokenizer


In [42]:
from spacy.lang.ru import Russian

In [72]:
def tokenize_X_spacy(X):
    X_new = []
    for item in X:
        text = item[0] + ' ' + item[1]
        doc = nlp(text)
        X_new.append([[token.text for token in doc]])
        return np.array(X_new)

Остальное не делал :(

N-Gram модели текстовой классификации
* Признаки - mystem
* n-gramm'ы - несколько слов подряд объединяются в один токен
* Модели - только логистическая регрессия
* Настоятельно рекомендуется использовать sparse-матрицы

Использовать Vowpal Wabbit вместо sklearn
* Признаки - обычный BoW title+description
* Главный вызов - заставить работать библиотеку и подготовить признаки